In [25]:
#Imports
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import re
from collections import Counter
import requests
print("complete")

complete


In [26]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    return text.split()

def build_vocab(word_list, vocab_size=5000):
    most_common = Counter(word_list).most_common(vocab_size - 1)
    vocab = {w: i+1 for i, (w, _) in enumerate(most_common)}
    vocab["<UNK>"] = 0
    return vocab

def encode_words(word_list, vocab):
    return [vocab.get(word, vocab["<UNK>"]) for word in word_list]

url = "https://www.gutenberg.org/files/11/11-0.txt"
response = requests.get(url)
raw_text = response.text

words = preprocess_text(raw_text)
vocab = build_vocab(words)               
encoded = encode_words(words, vocab)
idx2word = {i: w for w, i in vocab.items()}  

In [27]:

class SimpleGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(SimpleGRU, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.W_z = nn.Linear(input_dim, hidden_dim)
        self.U_z = nn.Linear(hidden_dim, hidden_dim, bias=False)

        self.W_r = nn.Linear(input_dim, hidden_dim)
        self.U_r = nn.Linear(hidden_dim, hidden_dim, bias=False)

        self.W_h = nn.Linear(input_dim, hidden_dim)
        self.U_h = nn.Linear(hidden_dim, hidden_dim, bias=False)

    def forward(self, x, h_0=None):
        seq_len, batch_size, _ = x.size()
        if h_0 is None:
            h_t = torch.zeros(batch_size, self.hidden_dim, device=x.device)
        else:
            h_t = h_0

        outputs = []
        for t in range(seq_len):
            x_t = x[t]
            z_t = torch.sigmoid(self.W_z(x_t) + self.U_z(h_t))
            r_t = torch.sigmoid(self.W_r(x_t) + self.U_r(h_t))
            h_tilde = torch.tanh(self.W_h(x_t) + self.U_h(r_t * h_t))
            h_t = (1 - z_t) * h_t + z_t * h_tilde
            outputs.append(h_t.unsqueeze(0))
        return torch.cat(outputs, dim=0), h_t


class GRULanguageModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(GRULanguageModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = SimpleGRU(embedding_dim, hidden_dim)
        self.decoder = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_seq):
        embedded = self.embedding(input_seq)
        gru_out, _ = self.gru(embedded)
        logits = self.decoder(gru_out[-1])
        return logits


In [ ]:
class NeuralLM(nn.Module):
    def __init__(self, vocab_size, embed_dim=64, hidden_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.GRU(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1]) 
        return out

vocab_size = len(vocab)
model = NeuralLM(vocab_size)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()
print("complete")

complete


In [29]:

from torch.utils.data import Dataset, DataLoader

class LanguageModelDataset(Dataset):
    def __init__(self, data, seq_len):
        self.data = data
        self.seq_len = seq_len

    def __len__(self):
        return len(self.data) - self.seq_len

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx : idx + self.seq_len])
        y = torch.tensor(self.data[idx + self.seq_len])
        return x, y


dataset = LanguageModelDataset(encoded, seq_len=5)
loader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)

In [30]:
for epoch in range(10):
    total_loss = 0
    for inputs, targets in loader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")
    torch.save(model.state_dict(), f"neural_lm_epoch{epoch+1}.pt")
print("complete")

Epoch 1, Loss: 2427.7701
Epoch 2, Loss: 1923.4140
Epoch 3, Loss: 1504.4037
Epoch 4, Loss: 1133.1184
Epoch 5, Loss: 862.7393
Epoch 6, Loss: 668.4510
Epoch 7, Loss: 534.2022
Epoch 8, Loss: 431.4213
Epoch 9, Loss: 350.7859
Epoch 10, Loss: 301.3509
complete


In [31]:
seq_len = 5 

In [32]:
def predict_next_word(seed_text):
    model.eval()
    seed_words = preprocess_text(seed_text)[-seq_len:]
    encoded_input = encode_words(seed_words, vocab)
    if len(encoded_input) < seq_len:
        encoded_input = [0] * (seq_len - len(encoded_input)) + encoded_input
    input_tensor = torch.tensor([encoded_input])
    with torch.no_grad():
        output = model(input_tensor)
        next_word_id = torch.argmax(output, dim=-1).item()
    return idx2word[next_word_id]

print("Input:", "she was not a bit")
print("Next word prediction:", predict_next_word("she was not a bit"))
print("complete")

Input: she was not a bit
Next word prediction: hurt
complete


In [33]:
print("Input:", "she was not a bit")
print("Next word prediction:", predict_next_word("she was not a bit"))

Input: she was not a bit
Next word prediction: hurt


In [34]:
print("Input:", "Alice fell down the rabbit")
print("Next word prediction:", predict_next_word("Alice fell down the rabbit"))

Input: Alice fell down the rabbit
Next word prediction: went


In [35]:
print("Input:", "I am going to attack")
print("Next word prediction:", predict_next_word("I am going to attack"))

Input: I am going to attack
Next word prediction: then
